**The training has been done on my desktop with 64gb RAM with 8 cores,therefore I'm not sure if colab will withstand it. I'm almost sure that it'll crash. I recommend runnig it on a CPU with atleast 64gb RAM**

*dumping all the pickled lists in a list called "final_embeddings"*

In [ ]:
final_embeddings = []
mynewlist = []

In [ ]:
import pickle

All the files opened using pickle can be obtained from the Feature_generation nb.

In [ ]:
with open('ESM1BT1.pkl', 'rb') as f:
  mynewlist = pickle.load(f)
final_embeddings.extend(mynewlist)


In [ ]:
with open('ESM1BT2.pkl', 'rb') as f:
  mynewlist = pickle.load(f)
final_embeddings.extend(mynewlist)


In [ ]:
with open('ESM1BT3.pkl', 'rb') as f:
  mynewlist = pickle.load(f)
final_embeddings.extend(mynewlist)


In [ ]:
with open('ESM1BT4.pkl', 'rb') as f:
  mynewlist = pickle.load(f)
final_embeddings.extend(mynewlist)


In [ ]:
with open('ESM1BT5.pkl', 'rb') as f:
  mynewlist = pickle.load(f)
final_embeddings.extend(mynewlist)


In [ ]:
with open('ESM1BT6.pkl', 'rb') as f:
  mynewlist = pickle.load(f)
final_embeddings.extend(mynewlist)


In [ ]:
with open('ESM1BT7.pkl', 'rb') as f:
  mynewlist = pickle.load(f)
final_embeddings.extend(mynewlist)


In [ ]:
with open('ESM1BT8.pkl', 'rb') as f:
  mynewlist = pickle.load(f)
final_embeddings.extend(mynewlist)


In [ ]:
len(final_embeddings)  #the length of this list should ome out to be 858777(equal to the number of sequences in the train.csv)

In [ ]:
import pandas as pd
d = pd.read_csv('Train.csv')
d.head()

In [ ]:
d['features'] = final_embeddings

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
d['LABEL'] = le.fit_transform(d['LABEL'])

In [ ]:
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(d, test_size=0.06, random_state=42, stratify=d[['LABEL']])

In [ ]:
training_embeddings = train.features.values
training_identifiers = train.SEQUENCE_ID.values
training_labels = train.LABEL.values

testing_embeddings = test.features.values
testing_identifiers = test.SEQUENCE_ID.values
testing_labels = test.LABEL.values

assert(len(training_identifiers) == len(training_embeddings))
assert(len(testing_identifiers) == len(testing_embeddings))

In [ ]:
from sklearn.preprocessing import StandardScaler  
scaler = StandardScaler()  
scaler.fit(list(training_embeddings))  
training_embeddings = scaler.transform(list(training_embeddings))  
# applying same transformation to test data
testing_embeddings = scaler.transform(list(testing_embeddings)) 

In [ ]:
from sklearn.ensemble import BaggingClassifier


The cell below(training process) took ~30 hours to finish. **See Extra in the bottom of the notebook**

In [ ]:
clf = BaggingClassifier(base_estimator=MLPClassifier(hidden_layer_sizes = 865, solver='sgd',early_stopping = True, n_iter_no_change = 13, alpha= 0.009, random_state=10,verbose = True, max_iter=10000),
                        n_estimators=10,verbose = 3, random_state=0).fit(list(training_embeddings), training_labels)

In [ ]:
from joblib import dump, load
dump(clf, 'esm1b1.joblib') 

After the training finished, the trained model can be found named esm1b1

In [ ]:
# clf = load('esm1b1.joblib') 

In [ ]:
predicted_testing_labels = clf.predict(testing_embeddings)
accuracy = accuracy_score(testing_labels, predicted_testing_labels)
print(f"The model has an accuracy of {accuracy:.2}")

**Submission**

Re-initialize


```
final_embeddings 
mynewlist
```



In [ ]:
final_embeddings = []
mynewlist = []

In [ ]:
with open('ESMTest1a.pkl', 'rb') as f:
  mynewlist = pickle.load(f)
final_embeddings.extend(mynewlist)

In [ ]:
with open('ESMTest1b.pkl', 'rb') as f:
  mynewlist = pickle.load(f)
final_embeddings.extend(mynewlist)

In [ ]:
with open('ESMTest2.pkl', 'rb') as f:
  mynewlist = pickle.load(f)
final_embeddings.extend(mynewlist)

In [ ]:
with open('ESMTest3a.pkl', 'rb') as f:
  mynewlist = pickle.load(f)
final_embeddings.extend(mynewlist)

In [ ]:
with open('ESMTest3b.pkl', 'rb') as f:
  mynewlist = pickle.load(f)
final_embeddings.extend(mynewlist)

In [ ]:
len(final_embeddings)  #the length of this list should ome out to be 253146(equal to the number of sequences in the Test.csv)

In [ ]:
final_embeddings = scaler.transform(list(final_embeddings))  

In [ ]:
predicted_testing_labelss = clf.predict(list(final_embeddings))

In [ ]:
predicted_testing_labelss = le.inverse_transform(predicted_testing_labelss)

In [ ]:
sub = pd.read_csv('SampleSubmission.csv')

In [ ]:
sub.drop(columns=['LABEL'],inplace = True)

In [ ]:
sub['LABEL'] = predicted_testing_labelss

In [ ]:
sub.to_csv('sub.csv',index = False)

**Extra**

The Classfier used above takes forever to train and doesn't substantially boost the performace(only 0.0003,0.03%) as compared to the previous solution using just a Single estimator. 

In [ ]:
multilayerperceptron = MLPClassifier(solver='adam',early_stopping = True, n_iter_no_change = 40, alpha= 0.008, random_state=10,verbose = True, max_iter=10000)

In [ ]:
parameters = {
    'hidden_layer_sizes': [(865,)]}

This model train in 3-5 hours max.

In [ ]:
classifiers = GridSearchCV(multilayerperceptron,parameters,verbose=3, cv=2, scoring="accuracy")
classifiers.fit(list(training_embeddings), training_labels)
clf = classifiers.best_estimator_